In [32]:
from CheckGraphic import CheckGraphicID

import warnings
warnings.filterwarnings("ignore")
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import pymongo
import bson
import matplotlib.pyplot as plt
import matplotlib
import random
from itertools import permutations
from IPython.display import Image 
from IPython.core.display import Image, display
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags

In [33]:
# for i in db_client["tweets_pipeline_v2"].find({"$or": [{"status":"pipelined"},
# #                                                            {"status":"graphicone_feed"}
#                                                       ],
#                                                        'source': 'sec',
# #                                                    'approved_timestamp': {'$gte': end_date, '$lte': start_date},
#                                                    }):
#     print(i['_id'])
#     print(i['entity_prepared']+i["bert_tags"])    
#     CheckGraphicID(i['_id'])



In [34]:
sources = []
for i in db_client["tweets_pipeline_v2"].find({"status":"pipelined"}):
    sources.append(i['source'])

In [36]:
Counter(sources)

Counter({'twitter': 4714,
         'instagram': 2,
         'yandex': 409,
         'def': 981,
         'google': 2815,
         'dailyshot': 898})

In [21]:
urls = []
for i in db_client["tweets_pipeline_v2"].find({"status":"pipelined" ,"source":"dailyshot"}):
#     print(i["img_urls"])
    urls.append([i["_id"], i["img_urls"]])

In [22]:
df = pd.DataFrame(urls, columns = ["id", "url"])
df

,id,url
0,5f450d3dddf689a3a18b3b5e,https://si.wsj.net/public/resources/images/B3-...
1,5f450d3d749d3b6b7842d192,https://si.wsj.net/public/resources/images/B3-...
2,5f450d438802ddfa3bbe76b1,https://si.wsj.net/public/resources/images/B3-...
3,5f450d44c25f9fb609aa12bb,https://si.wsj.net/public/resources/images/B3-...
4,5f450d46880af837dc0b4e2a,https://si.wsj.net/public/resources/images/B3-...
...,...,...
893,5f4a5f2f69634a2626244016,https://si.wsj.net/public/resources/images/B3-...
894,5f4a5f2f6e9cb7b440173325,https://si.wsj.net/public/resources/images/B3-...
895,5f4a5f325f6b140a062f25d3,https://si.wsj.net/public/resources/images/B3-...
896,5f4a5f330ea8afc6d025c575,https://si.wsj.net/public/resources/images/B3-...


In [23]:
print(len(Counter(df.url)))
Counter(df.url)

898


Counter({'https://si.wsj.net/public/resources/images/B3-GJ810_Dshot_NS_20200326050549.png': 1,
         'https://si.wsj.net/public/resources/images/B3-GJ858_Dshot_NS_20200326050610.png': 1,
         'https://si.wsj.net/public/resources/images/B3-GJ861_Dshot_NS_20200326050611.png': 1,
         'https://si.wsj.net/public/resources/images/B3-GJ827_Dshot_NS_20200326050557.png': 1,
         'https://si.wsj.net/public/resources/images/B3-GJ805_Dshot_NS_20200326050547.png': 1,
         'https://si.wsj.net/public/resources/images/B3-GJ791_Dshot_NS_20200326050541.png': 1,
         'https://si.wsj.net/public/resources/images/B3-GO426_Dshot_NS_20200423050323.png': 1,
         'https://si.wsj.net/public/resources/images/B3-GO423_Dshot_NS_20200423050320.png': 1,
         'https://si.wsj.net/public/resources/images/B3-GO422_Dshot_NS_20200423050319.png': 1,
         'https://si.wsj.net/public/resources/images/B3-GO364_Dshot_NS_20200423050220.png': 1,
         'https://si.wsj.net/public/resources/imag

In [24]:
df

,id,url
0,5f450d3dddf689a3a18b3b5e,https://si.wsj.net/public/resources/images/B3-...
1,5f450d3d749d3b6b7842d192,https://si.wsj.net/public/resources/images/B3-...
2,5f450d438802ddfa3bbe76b1,https://si.wsj.net/public/resources/images/B3-...
3,5f450d44c25f9fb609aa12bb,https://si.wsj.net/public/resources/images/B3-...
4,5f450d46880af837dc0b4e2a,https://si.wsj.net/public/resources/images/B3-...
...,...,...
893,5f4a5f2f69634a2626244016,https://si.wsj.net/public/resources/images/B3-...
894,5f4a5f2f6e9cb7b440173325,https://si.wsj.net/public/resources/images/B3-...
895,5f4a5f325f6b140a062f25d3,https://si.wsj.net/public/resources/images/B3-...
896,5f4a5f330ea8afc6d025c575,https://si.wsj.net/public/resources/images/B3-...


In [25]:
dublDF = df[df.duplicated(subset=["url"], keep='first')]
dublDF

,id,url


In [26]:
len(set(df.url) - set(dublDF.url))

898

In [27]:
df[df['url'] == 'https://si.wsj.net/public/resources/images/B3-BL796_Dshot_NS_20180820030924.png']

,id,url
890,5f4a5d518fa0940f9389fc43,https://si.wsj.net/public/resources/images/B3-...


In [28]:
from tqdm import tqdm

In [29]:
from bson import ObjectId

In [30]:
db_client["tweets_pipeline_v2"].find({"status":"pipelined" ,"source":"dailyshot"}).count()

898

In [31]:
for i in tqdm(dublDF.id, total = len(dublDF.id)):
    db_client["tweets_pipeline_v2"].delete_one({'_id': ObjectId(i)})

0it [00:00, ?it/s]


In [6]:
import urllib.request
from PIL import Image
# url = 'http://www.example.com/my_image_is_not_your_image.png'
# image = Image.open(urllib.request.urlopen(url))

In [26]:
def sizeFilter(size):
    width, height = image.size
    if width < 300 and height < 175:
        return False
    else:
        return True

In [24]:
(325, 260)
(337, 218)
height > 175

(337, 218)

In [1]:
for i in db_client["tweets_pipeline_v2"].find({"$or": [{"status":"trasher"},
#                                                            {"status":"graphicone_feed"}
                                                      ],
                                                       'source': 'sec',
#                                                    'approved_timestamp': {'$gte': end_date, '$lte': start_date},
                                                   }):
    

    image = Image.open(urllib.request.urlopen(i['img_urls']))
    if sizeFilter(image.size):
        print(image.size)
        print(i['_id'])
        print(i["bert_tags"])
#         print(i['entity_prepared']+i["bert_tags"])    
        CheckGraphicID(i['_id'])
#     break

In [ ]:
for i in db_client["tweets_pipeline_v2"].find({"$or": [{"status":"trasher"},
#                                                            {"status":"graphicone_feed"}
                                                      ],
                                                       'source': 'sec',
#                                                    'approved_timestamp': {'$gte': end_date, '$lte': start_date},
                                                   }):
    

    i['img_urls']

In [ ]:
def dowloadGraphic()